**laod features**

In [20]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

#import models
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import NMF 
from sklearn.neighbors import KNeighborsClassifier

#Test:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

plt.style.use('ggplot')
# data processing libraries
import numpy as np
import pandas as pd

# plotting libraries
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

# machine-learning libraries
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise
#from scipy.sparse import csr_matrix

# miscellaneous
import pickle

import numpy as np
from sklearn.decomposition import NMF 
import pandas as pd

# data processing libraries
import numpy as np
import pandas as pd

# plotting libraries
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

# machine-learning libraries
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise
#from scipy.sparse import csr_matrix

# miscellaneous
import pickle

**load data**

In [21]:
movies = pd.read_csv('data/movies.csv')
rating = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')
links = pd.read_csv('data/links.csv')

In [22]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


**create R matrix**

In [23]:
Rt

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,3.0,4.5,4.0,0.0,3.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,0.0,4.5,4.5,3.0,0.0,4.0,0.0,3.0,...,0.0,0.5,0.0,0.0,4.5,0.0,2.5,1.0,1.5,0.0


In [24]:
# calculate the number of ratings per movie
rating_count = rating.groupby('movieId')[['rating']].count()

# filter for movies with more than 20 ratings and extract the index
popular_movies = rating_count[rating_count['rating']>20].index

# filter the ratings matrix and only keep the popular movies
dfn = rating[rating['movieId'].isin(popular_movies)].copy()

# need to remake user ids and movie ids since they are not sequential
user_ids = dfn['userId'].unique()
user_id_map = {v:k for k,v in enumerate(user_ids)} #reduce with -1 user Id
dfn['userId'] = dfn['userId'].map(user_id_map)


movie_ids = dfn['movieId'].unique()
movie_id_map = {v:k for k,v in enumerate(movie_ids)} 
dfn['movieId'] = dfn['movieId'].map(movie_id_map)

# Initialize a sparse user-item rating matrix
# (data, (row_ind, col_ind) from scipy.sparse import csr_matrix
#careful csr fills NaN=0
R = csr_matrix((dfn['rating'], (dfn['userId'], dfn['movieId']))) 

#if all is well, the shape of the user-item matrix R will be (610, 1235)


#the movies.csv file should also the transformed accordingly

# filter out unpopular movies
movies = movies[movies['movieId'].isin(movie_ids)]
# redefine movie ids
movies['movieId'] = movies['movieId'].map(movie_id_map)

**show R matrix**

In [25]:
#get a column with all userIds
user_grouping = rating.groupby(['userId'])['rating'].mean()

In [26]:
R.todense()[0,:24]

matrix([[4., 4., 4., 5., 5., 3., 5., 4., 5., 5., 5., 3., 5., 4., 5., 3.,
         3., 5., 4., 4., 5., 4., 4., 5.]])

In [27]:
print(R)

  (0, 0)	4.0
  (0, 1)	4.0
  (0, 2)	4.0
  (0, 3)	5.0
  (0, 4)	5.0
  (0, 5)	3.0
  (0, 6)	5.0
  (0, 7)	4.0
  (0, 8)	5.0
  (0, 9)	5.0
  (0, 10)	5.0
  (0, 11)	3.0
  (0, 12)	5.0
  (0, 13)	4.0
  (0, 14)	5.0
  (0, 15)	3.0
  (0, 16)	3.0
  (0, 17)	5.0
  (0, 18)	4.0
  (0, 19)	4.0
  (0, 20)	5.0
  (0, 21)	4.0
  (0, 22)	4.0
  (0, 23)	5.0
  (0, 24)	4.0
  :	:
  (609, 1185)	3.0
  (609, 1186)	5.0
  (609, 1188)	5.0
  (609, 1189)	5.0
  (609, 1190)	3.0
  (609, 1192)	3.5
  (609, 1193)	4.0
  (609, 1194)	4.0
  (609, 1195)	4.0
  (609, 1196)	4.0
  (609, 1197)	3.0
  (609, 1199)	4.5
  (609, 1200)	5.0
  (609, 1203)	3.5
  (609, 1204)	3.0
  (609, 1211)	2.5
  (609, 1213)	3.5
  (609, 1214)	2.0
  (609, 1215)	4.0
  (609, 1216)	3.5
  (609, 1218)	4.0
  (609, 1219)	4.0
  (609, 1227)	4.0
  (609, 1231)	2.5
  (609, 1232)	3.0


In [46]:
#give titles to box axes and get a panda frame
Rt = pd.DataFrame(data=R.todense(), columns=movies['title'],index = user_grouping.index)
Rt.head(3)
Rt.to_csv('my_Rt.csv', index=False)

## A. Nearest Neighbors model 

**fit model**

In [29]:
modelnn = NearestNeighbors(metric='cosine')

# fit it to the Ratings matrix
modelnn.fit(Rt)

NearestNeighbors(metric='cosine')

In [30]:
def recommend_nn(query, model, Rt, k=10):
    """
    Filters and recommends the top k movies for any given input query based on a trained NMF model. 
    Returns a list of k movie ids.
    """
        
    # 1. construct new_user-item dataframe given the query
    new_user_dataframe =  pd.DataFrame(query, columns=movies['title'], index=['new_user'])
    # 1.2. fill the NaN
    new_user_dataframe_imputed = new_user_dataframe.fillna(0) #better mean
    # 2. scoring
    # calculates the distances to all other users in the data!
    similarity_scores, neighbor_ids = model.kneighbors(
    new_user_dataframe_imputed,
    n_neighbors=15,
    return_distance=True
    )

    # sklearn returns a list of predictions
    # extract the first and only value of the list

    neighbors_df = pd.DataFrame(
    data = {'neighbor_id': neighbor_ids[0], 'similarity_score': similarity_scores[0]}
    )
    
    # 3. ranking
    # only look at ratings for users that are similar!
    neighborhood = Rt.iloc[neighbor_ids[0]]
  
    
        # filter out movies already seen by the user
    neighborhood_filtered = neighborhood.drop(query.keys(),axis=1)
   

    # calculate the summed up rating for each movie
    # summing up introduces a bias for popular movies
    # averaging introduces bias for movies only seen by few users in the neighboorhood

    df_score = neighborhood_filtered.sum().sort_values(ascending=False)
    
    # return the top-k highest rated movie ids or titles
    df_score_ranked = df_score.sort_values(ascending=False).index.tolist()
    recommended = df_score_ranked[:k]
    return recommended, df_score

## B. NMF model

In [31]:
nmf_model = NMF(n_components=100, max_iter=1000, tol=0.001)

# fit it to the Ratings matrix
nmf_model.fit(Rt)

NMF(max_iter=1000, n_components=100, tol=0.001)

**create Q**

In [32]:
Q_matrix = nmf_model.components_
Q = pd.DataFrame(Q_matrix, columns=movies['title'], index=nmf_model.get_feature_names_out())
Q.head(2)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
nmf0,0.078029,0.0,0.000000,1.930389,0.000000,0.0,0.0,4.522229,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nmf1,0.018950,0.0,2.379092,2.424021,5.878973,0.0,0.0,1.786681,3.033053,1.000632,...,0.0,0.249983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
P_matrix = nmf_model.transform(R)

C:\Users\mlomm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  warnings.warn(


In [34]:
# to dataframe for better visualization
P = pd.DataFrame(
    data=P_matrix, 
    index=user_ids, 
    columns=nmf_model.get_feature_names_out()
)
P.head(3)

,nmf0,nmf1,nmf2,nmf3,nmf4,nmf5,nmf6,nmf7,nmf8,nmf9,...,nmf90,nmf91,nmf92,nmf93,nmf94,nmf95,nmf96,nmf97,nmf98,nmf99
1,0.009845,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000
2,0.000000,0.0,0.0,0.040579,0.0,0.0,0.0,0.002222,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.32428,0.444198,0.000000
3,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.025962,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.191917


In [35]:
R_hat_matrix = P.dot(Q)
R_hat_matrix.head(3)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
1,4.117034,3.985199,3.594906,5.140483,5.347635,2.702827,4.789465,3.910291,4.846142,5.007037,...,0.000000,0.000482,0.000000,0.000969,0.000000,0.000000,0.000000,0.016703,0.027047,0.091550
2,0.174618,0.000000,0.046827,0.517513,0.940494,0.042780,0.021416,0.210554,0.023883,0.097196,...,0.032919,0.002804,0.058260,0.016157,0.130586,0.000000,0.119772,0.035644,0.016019,0.000971
3,0.150085,0.049013,0.077612,0.093899,0.163491,0.056053,0.061512,0.102771,0.029983,0.049368,...,0.026984,0.014413,0.002704,0.051018,0.017581,0.039346,0.002339,0.001862,0.050564,0.001031


In [36]:
R_hat = pd.DataFrame(
    data=R_hat_matrix, 
    index=user_grouping.index, 
    columns=movies['title']
)
R_hat.head(3)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.117034,3.985199,3.594906,5.140483,5.347635,2.702827,4.789465,3.910291,4.846142,5.007037,...,0.000000,0.000482,0.000000,0.000969,0.000000,0.000000,0.000000,0.016703,0.027047,0.091550
2,0.174618,0.000000,0.046827,0.517513,0.940494,0.042780,0.021416,0.210554,0.023883,0.097196,...,0.032919,0.002804,0.058260,0.016157,0.130586,0.000000,0.119772,0.035644,0.016019,0.000971
3,0.150085,0.049013,0.077612,0.093899,0.163491,0.056053,0.061512,0.102771,0.029983,0.049368,...,0.026984,0.014413,0.002704,0.051018,0.017581,0.039346,0.002339,0.001862,0.050564,0.001031


In [37]:
nmf_model.reconstruction_err_

551.8939603016127

In [38]:
def recommend_nmf(query, model, k=10):
    """
    Filters and recommends the top k movies for any given input query based on a trained NMF model. 
    Returns a list of k movie ids.
    """
    
    # 1. construct new_user-item dataframe given the query(votings of the new user)
   
    new_user_dataframe = pd.DataFrame(query, columns=movies['title'], index=['new_user'])
   
    new_user_dataframe_imputed =new_user_dataframe.fillna(0)

    P_new_user_matrix = model.transform(new_user_dataframe_imputed)
    # get as dataframe for a better visualizarion
    P_new_user = pd.DataFrame(P_new_user_matrix, 
                         columns = model.get_feature_names_out(),
                         index = ['new_user'])
    R_hat_new_user_matrix = np.dot(P_new_user,Q)
    # get as dataframe for a better visualizarion
    R_hat_new_user = pd.DataFrame(data=R_hat_new_user_matrix,
                         columns=movies['title'],
                         index = ['new_user'])
    R_hat_new_filtered = R_hat_new_user#.drop(new_user_query.keys(), axis=1)
    R_hat_new_filtered.T.sort_values(by=["new_user"], ascending=False).index.tolist()
    ranked =  R_hat_new_filtered.T.sort_values(by=["new_user"], ascending=False).index.tolist()
    # 2. scoring
    
        # calculate the score with the NMF model
    
    
    # 3. ranking
    
        # filter out movies already seen by the user
    
        # return the top-k highest rated movie ids or titles
    
    recommended = ranked[:k]
    return recommended, #R_hat_new_filtered.T.sort_values(by=["new_user"], ascending=False)

## New user prediction

In [39]:
new_user_query = {"Toy Story (1995)": 5,
                 "Grumpier Old Men (1995)":2,
                 "Casino (1995)":3.5,
                 "Sabrina (1995)":4,
                 "GoldenEye (1995)":5}
print(new_user_query)

{'Toy Story (1995)': 5, 'Grumpier Old Men (1995)': 2, 'Casino (1995)': 3.5, 'Sabrina (1995)': 4, 'GoldenEye (1995)': 5}


In [40]:
new_user_dataframe = pd.DataFrame(new_user_query, columns=movies['title'], index=['new_user'])
new_user_dataframe

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),GoldenEye (1995),"American President, The (1995)",Casino (1995),Sense and Sensibility (1995),...,Captain America: Civil War (2016),Doctor Strange (2016),The Martian (2015),Inside Out (2015),The Revenant (2015),"Big Short, The (2015)",Zootopia (2016),Arrival (2016),Rogue One: A Star Wars Story (2016),Logan (2017)
new_user,5,NaN,2,NaN,NaN,4,5,NaN,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
recommend_nmf(new_user_query, nmf_model, k=10)

(['Toy Story (1995)',
  "Charlotte's Web (1973)",
  'Big Lebowski, The (1998)',
  'Down Periscope (1996)',
  'U.S. Marshals (1998)',
  'Lethal Weapon 3 (1992)',
  'Rumble in the Bronx (Hont faan kui) (1995)',
  'Die Hard: With a Vengeance (1995)',
  'Taxi Driver (1976)',
  'Bridges of Madison County, The (1995)'],)

In [42]:
recommend_nn(new_user_query, modelnn, Rt)

(['Ransom (1996)',
  'Stand by Me (1986)',
  'Happy Gilmore (1996)',
  'Broken Arrow (1996)',
  'Down Periscope (1996)',
  'Beauty and the Beast (1991)',
  'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
  'Die Hard: With a Vengeance (1995)',
  'Taxi Driver (1976)',
  'Heat (1995)'],
 title
 Ransom (1996)                                  33.0
 Stand by Me (1986)                             32.0
 Happy Gilmore (1996)                           31.0
 Broken Arrow (1996)                            27.0
 Down Periscope (1996)                          27.0
                                                ... 
 Big Daddy (1999)                                0.0
 Arachnophobia (1990)                            0.0
 South Park: Bigger, Longer and Uncut (1999)     0.0
 Wild Wild West (1999)                           0.0
 Logan (2017)                                    0.0
 Length: 1230, dtype: float64)

In [43]:
with open('./modelnn.pkl', 'wb') as file:
    pickle.dump(modelnn, file)

In [44]:
with open('./nmf_mode.pkl', 'wb') as file:
    pickle.dump(nmf_model, file)

In [45]:
print(movies)

      movieId                                title   
0           0                     Toy Story (1995)  \
1         325                       Jumanji (1995)   
2           1              Grumpier Old Men (1995)   
4         326   Father of the Bride Part II (1995)   
5           2                          Heat (1995)   
...       ...                                  ...   
9162      808                Big Short, The (2015)   
9223      643                      Zootopia (2016)   
9392      809                       Arrival (2016)   
9433      644  Rogue One: A Star Wars Story (2016)   
9463      810                         Logan (2017)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
4                                          Comedy  
5                           Action|Crime|Thriller  
...                                    